# Import Modules

In [1]:
from utils import simple_analysis, dataset_generator
from model import dense_basic
from torch.utils.data.dataloader import DataLoader
from torch import optim, nn, Tensor, LongTensor
from tqdm import tqdm
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

print("PyTorch version:[%s]." % (torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device:[%s]." % (device))

PyTorch version:[1.8.1].
device:[cpu].
/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554786529/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


# EDA

In [ ]:
simple_analysis.run("/opt/ml/input/data")

# Load Data

In [ ]:
train_set = dataset_generator.generate_train_datasets("/opt/ml/input/data")
train_set_loader = DataLoader(train_set, batch_size=256)

# Model Define

In [7]:
model = dense_basic.DenseBasic().to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

if True:
    model.load_state_dict(torch.load("./result/last_model"))

# Training

In [ ]:
def convert_label(labels):
    results = []
    for label in labels:
        result = 0
        if label[2] == 0:
            result = 12
        elif label[3] == 0:
            result = 6
        else:
            result = 0
        
        if label[0] == 1:
            result += 3
        
        result += label[1]

        # results.append([1 if i == result else 0 for i in range(18)])
        results.append(result)

    return LongTensor(results)

In [ ]:
running_loss = 0.0
model.eval()
for index, (batch_source, batch_label) in tqdm(enumerate(train_set_loader), total=len(train_set_loader)):
    source = batch_source.to(device)
    labels = convert_label(batch_label).to(device)

    optimizer.zero_grad()

    outputs = model(source)
    
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if index % 5 == 4:
        print()
        print(f'loss: {running_loss / 5}')
        running_loss = 0.0
    
    if index % 25 == 24:
        torch.save(model.state_dict(), "./result/last_model")
        print("Model Saved")
    
torch.save(model.state_dict(), "./result/last_model")

# Test

In [2]:
test_dataset = dataset_generator.generate_test_dataset("/opt/ml/input/data")
test_set_loader = DataLoader(test_dataset)

100%|██████████| 12600/12600 [00:35<00:00, 354.31it/s]


In [8]:
import pandas as pd
result = pd.DataFrame({"ImageID": [], "ans": []})
for index, (batch_source, batch_paths) in tqdm(enumerate(test_set_loader)):
    source = batch_source.to(device)
    
    outputs = model(source)
    print(torch.argmax(output, dim=0))
    break
    


0it [01:31, ?it/s]


KeyboardInterrupt: 